In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pickle
import random
from PIL import Image, ImageEnhance, ImageFilter

In [2]:
train_path = './data/train.txt'
test_path = './data/test.txt'
val_path = './data/val.txt'

train_lines = open(train_path, 'r').readlines()
test_lines = open(test_path, 'r').readlines()
val_lines = open(val_path, 'r').readlines()

size_1 = 500
size_2 = 250

labels = ['normal', 'pneumonia','covid']

In [3]:
def enhance(img):
    """
    To enhance the image
    """
    img_enhanced = ImageEnhance.Sharpness(img).enhance(2.0)
    img_enhanced = ImageEnhance.Color(img_enhanced).enhance(2.0)
    img_enhanced = ImageEnhance.Contrast(img_enhanced).enhance(2.0)
    img_enhanced = img_enhanced.filter(ImageFilter.DETAIL)
    img_enhanced = img_enhanced.filter(ImageFilter.EDGE_ENHANCE_MORE)
    img_enhanced = img_enhanced.filter(ImageFilter.SMOOTH_MORE)

    return img_enhanced

def create_data(target_lines):
    """
    Most of the hardwork are here. Read, create, enhance, etc.
    """
    data = []
    for line in target_lines:
        path, label = line.split(' ')
        label = int(label)
        try :
            img_arr = Image.open(path).convert('L')
            # Enhance Image
            enhanced = enhance(img_arr)
            # resize to 500x500
            enhanced = enhanced.resize((size_1, size_1), Image.BICUBIC)
            # crop (unsless its a covid image)
            if path.find('covid-chestxray-dataset') == -1:
                left = 50
                top = 50
                right = 450
                bottom = 450
                crop_image = enhanced.crop((left, top, right, bottom))
            # resize to 250x250 and make it to an array
            new_arr = crop_image.resize((size_2, size_2), Image.BICUBIC).getdata()
            # add to data
            data.append([new_arr, label])
        except Exception as e:
            print(e)
            print(path)

    return data

def prep_data(prep_lines):
    """
    Shuffle the data and adding the data and label to numpy array
    """
    data = create_data(prep_lines)
    random.shuffle(data)

    X = []
    y = []

    for features, label in data:
        X.append(features)
        y.append(label)

    X = np.array(X).reshape(-1, size_2, size_2, 1)
    y = np.array(y)

    return X,y

def save(arr, filename):
    """
    Saving numpy array to .pickle file
    """
    pickle_out = open(filename, 'wb')
    pickle.dump(arr, pickle_out)
    pickle_out.close()

In [4]:
# X_val, y_val = prep_data(val_lines)
# X_train, y_train = prep_data(train_lines)
# X_test, y_test = prep_data(test_lines)

In [5]:
i = 69
print(labels[y_train[i]])
plt.imshow(X_train[i].reshape(size_2, size_2), cmap='gray')
plt.show()

NameError: name 'y_train' is not defined

In [6]:
print(X_train.shape)

(5630, 250, 250, 1)


In [6]:
save(X_train, './arr_enhanced/X_train_enhanced.pickle')
save(y_train, './arr_enhanced/y_train_enhanced.pickle')